<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/Multilingual_Named_Entity_Recognation_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Multilingual Named Entity Recognition

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
!pip install --upgrade datasets transformers fsspec huggingface_hub

In [ ]:
from datasets import load_dataset
load_dataset('xtreme',name='PAN-X.de')

In [ ]:
from collections import defaultdict
from datasets import DatasetDict #dataset strukturu:train,valid,test kimi olan daatsetler ucun

langs = ["de", "fr", "it", "en"] #4 dil
fracs = [0.629, 0.229, 0.084, 0.059] #her dil ucun nece faiz data secilecek
panx_ch = defaultdict(DatasetDict)#hər bir dil üçün bir DatasetDict (train/validation/test) saxlayacaq.


#Bu kod, hər bir dil üçün PAN-X datasını yükləyir, train, validation, test hissələrini qarışdırır və onlardan yalnız müəyyən faiz (frac qədər)
# nümunə seçərək panx_ch adlı sözlükdə saxlayır.
for lang, frac in zip(langs, fracs):
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    for split in ds:
        panx_ch[lang][split] = (
            ds[split]
            .shuffle(seed=0)
            .select(range(int(frac * ds[split].num_rows))))

In [ ]:
panx_ch['de']['train'][0]

In [ ]:
panx_ch

In [ ]:
import pandas as pd
pd.DataFrame({lang:[panx_ch[lang]['train'].num_rows] for lang in langs}, #her language den nece example gelir
             index=['Number of training examples'])

In [ ]:
#Alman dilindəki train datasından ilk nümunənin tərkibində nə olduğunu (məsələn, hansı sözlər və etiketlər) göstərməkdir.
element=panx_ch['de']['train'][0]
for key,value in element.items():
    print(f'{key}:{value}')

In [ ]:
tags=panx_ch['de']['train'].features['ner_tags'].feature  #taglara baxiriq
tags
#features → Bu datasetdəki bütün sütunların (features) xüsusiyyətlərini (metadata) saxlayan obyekt.

In [ ]:
def create_tag_names(batch):
    return{'ner_tags_str':[tags.int2str(idx) for idx in batch['ner_tags']]}
panx_de=panx_ch['de'].map(create_tag_names)
#Alman dilindəki datasetdə olan ner_tags sütunundakı rəqəmləri onların müvafiq etiket adlarına
#(məsələn, B-PER, I-LOC, O və s.) çevirir və yeni ner_tags_str sütunu kimi əlavə edir.

In [ ]:
#hazirlamag idi gelen dersden modeli dzeldeceyik

# Multilingual Transformers

In [ ]:
from transformers import AutoTokenizer

bert_model_name='bert-base-cased' #uncased boyuk balaca herflere fikir vermir,cased fikir verir.
xlmr_model_name='xlm-roberta-base'# bertle eyni architektura var ama xlmr multilanguagedir coxlu dil ustunde trainolub ama bert ing dilde,
#xlmr-sentencepiece tokenizeer edir ama bert wordpiece
bert_tokenizer=AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer=AutoTokenizer.from_pretrained(xlmr_model_name)

In [ ]:
text='Jack Sparrow love New York'
bert_tokens=bert_tokenizer(text).tokens()
xlmr_tokens=xlmr_tokenizer(text).tokens()#senetnce piece <s> </s>
#normalization boyu-->balaca , eyni formata salir , boshluqlari silir,
#pretokenizer boshluqlara gore, xlm de yoxdu
#postprocessing bizim basha dusheeyimiz dilde

In [ ]:
df=pd.DataFrame([bert_tokens, xlmr_tokens],
                 index=['Bert','XLM-R']) #yanida hecne yoxdurss_
#robert-->robust fln
df

# SentencePiece Tokenizer

In [ ]:
''.join(xlmr_tokens).replace('\u2581'," ") #bu _ xettin kodud unicod \u2581

# Creating a Custom Model for Token Classification

In [ ]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers import XLMRobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel
from transformers import Trainer # Make sure Trainer is imported here if not already

In [ ]:
# Head--> Ber for ne uchun Body oyrenib Head 0 dan  , onu oyredirik, bizde HEAd olamsa?? HUggingFace 'token classificton'--body+head-NER-di

In [ ]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.roberta = XLMRobertaModel(config, add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None, num_items_in_batch=None, **kwargs):
        # Added num_items_in_batch to the forward method signature
        # and it will be captured by **kwargs before being passed to the internal model.
        # We don't need to explicitly use num_items_in_batch here for this model.
        outputs = self.roberta(input_ids, attention_mask=attention_mask, **kwargs)
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )

# Rest of your code remains the same
# You can now rerun the cell with trainer.train()

# Loading a Custom Model

In [ ]:
index2tag={idx:tag for idx,tag in enumerate(tags.names)}
tag2index={tag:idx for idx,tag in enumerate(tags.names)}

In [ ]:
tag2index

In [ ]:
index2tag

In [ ]:
tags.names

In [ ]:
from transformers import AutoConfig

xlmr_config=AutoConfig.from_pretrained(xlmr_model_name,num_labels=tags.num_classes,
                                       id2label=index2tag,label2id=tag2index)

In [ ]:
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xlmr_model=XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name,config=xlmr_config).to(device)

In [ ]:
#datani hazirla
input_ids=xlmr_tokenizer.encode(text, return_tensors='pt') #encode ne dirdi?? niye bele
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=['Tokens','Input IDs'])

In [ ]:
outputs=xlmr_model(input_ids.to(device)).logits
predictions=torch.argmax(outputs,dim=-1)
print(f'Number of tokens in sequence: {len(predictions[0])}')
print(f'Shape of outputs tensor: {outputs.shape}')

In [ ]:
outputs

In [ ]:
predictions

In [ ]:
preds=[tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens,preds],index=['Tokens','Tags'])

In [ ]:
tags.names

In [ ]:
def tag_text(text, tags, model, tokenizer):#kecendefeki
    tokens = tokenizer(text).tokens() #Bu isə, yuxarıdakı tokenizer nəticəsindən sadəcə tokenləri çıxarır (yəni [‘play’, ‘##ing’] kimi nəticə verir).
    input_ids = tokenizer.encode(text, return_tensors='pt')#reqem kim --Bu hissə text mətnini alır və onu token ID-lərinə çevirir.Yəni sözlər → subword tokenlər → rəqəmlər (ID-lər)
    outputs = model(input_ids.to(device))[0] #last_hidden_state
    predictions = torch.argmax(outputs, dim=-1) #calssi verir yeni BIO-da hansi calssa argmaxi coxdu--> bu
    preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])
# text = "Obama"
# tokens = ['O', '##bam', '##a']
# input_ids = [101, 1234, 5678, 102]
#outputs=outputs.shape = [batch_size, sequence_length, num_tags]
#predictions = [[0, 2, 2, 0, 4, 5, 0, 3, 3, 0]]
# predictions = [0, 2, 2, 1]
# tags.names = ['O', 'B-PER', 'I-PER']
# preds = ['O', 'I-PER', 'I-PER', 'B-PER']

In [ ]:
# 1. tokenizer(text)
    # Bu hissə, tokenizer obyektinə text adlı mətni verir və tokenizer onu emal edir.
    # Məsələn: text = "playing"
    # Tokenizer: WordPiece, Unigram və s.
    # Nəticə: Tokenizer obyektindən çıxan bir Encoded object (məs: Encoding).

# output = (
#     last_hidden_state,     # [0] → əsas nəticə (ən çox istifadə olunan)
#     pooled_output,         # [1] → CLS tokenin çıxışı (ən çox klassifikasiya üçün istifadə olunur)
#     hidden_states (optional),
#     attentions (optional)
# )

# Tokenizing Texts for NER

In [ ]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]],
['Tokens', 'Tags'])

In [ ]:
words, labels=de_example['tokens'], de_example['ner_tags']

In [ ]:
words

In [ ]:
tokenized_input=xlmr_tokenizer(words, is_split_into_words=True) #sozlere bolunu b yoxsa yox, string olara false
tokens=xlmr_tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
pd.DataFrame([tokens,labels], index=['Tokens','Labels'])#sozler bolunu  label ile solzer ust uste duhmur

In [ ]:
tokenized_input #etxt--reqem

In [ ]:
tokenized_input.word_ids()

In [ ]:
word_ids=tokenized_input.word_ids()
pd.DataFrame([tokens, word_ids], index=['Tokens','Word IDs'])# eyni soelr yeni reqemlerle, NONElere -100 deyeceyik,  pytorchda -100 ignore edir

In [ ]:
previous_word_idx=None
label_ids=[]

for word_idx in word_ids:
    if word_idx is None or word_idx==previous_word_idx:
        label_ids.append(-100)
    else:
        label_ids.append(labels[word_idx])
    previous_word_idx=word_idx

labels=[index2tag[l] if l!=-100 else 'IGN' for l in label_ids]
index=['Tokens','Word IDs','Label IDs','Labels']
df=pd.DataFrame([tokens, word_ids,label_ids, labels], index=index)
df

In [ ]:
label_ids

In [ ]:
index2tag

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=True
    )

    all_labels = []

    for i in range(len(examples["tokens"])):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                labels.append(-100)
            else:
                labels.append(examples["ner_tags"][i][word_idx])
            previous_word_idx = word_idx
        all_labels.append(labels)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs


def encode_panx_dataset(corpus):
    encoded_corpus = DatasetDict()
    for split, dataset in corpus.items():
        encoded_corpus[split] = dataset.map(
            tokenize_and_align_labels,
            batched=True,  # Doğrudur burada
            remove_columns=["langs", "ner_tags", "tokens"]
        )
    return encoded_corpus


panx_de_encoded = encode_panx_dataset(panx_ch["de"])
print(panx_de_encoded["train"][0])

In [ ]:
!pip install seqeval

In [ ]:
from seqeval.metrics import classification_report

y_true = [['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
y_pred = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]

# You can now use the corrected function:
print(classification_report(y_true, y_pred))

In [ ]:
import numpy as np #hugging face list ichind elist gozleyir
def align_predictions(predictions, label_ids):
    preds=np.argmax(predictions, axis=2)
    batch_size, seq_len=preds.shape
    label_list, preds_list=[],[]

    for batch_idx in range(batch_size):
        example_labels, example_preds=[],[]
        for seq_idx in range(seq_len):
            if label_ids[batch_idx, seq_idx]!=-100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])
        label_list.append(example_labels)
        preds_list.append(example_preds)
    return preds_list, label_list

# Fine-Tuning XLM-RoBerta

In [ ]:
from transformers import TrainingArguments

num_epochs=3
batch_size=24
logging_steps=len(panx_de_encoded['train'])//batch_size

model_name=f'{xlmr_model_name}-finetuned-panx-de'
training_args=TrainingArguments(
    output_dir=model_name, log_level='error', num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy='epoch',
    save_steps=1e6, #100 000 addimdan bir save etsin yeni etmesin  her iterasiyada  save etmek istemirk, istesek epoch yaz
    weight_decay=0.01,
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=True
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true=align_predictions(eval_pred.predictions, eval_pred.label_ids)
    return {'f1':f1_score(y_true, y_pred)} #butun duhmeldui ust uste

In [ ]:
from transformers import DataCollatorForTokenClassification #Ner uchun Pad etmeye, her defe ramd ayer tutur model , funkdiyani ichine yukleyek modlei ordan caqirsin

# Correct the typo in the class name
data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [ ]:
def model_init():
    return XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name,
                                                            config=xlmr_config).to(device)

In [ ]:
from transformers import Trainer

trainer=Trainer(model_init=model_init,
                args=training_args, #compile
                data_collator=data_collator,#nece pad
                compute_metrics=compute_metrics, #hani metrics
                train_dataset=panx_de_encoded['train'],
                eval_dataset=panx_de_encoded['validation'],
                tokenizer=xlmr_tokenizer)

In [ ]:
trainer.train()
# trainer.push_to_hub(config)

In [ ]:
# trainer.push_to_hub()

In [ ]:
df = pd.DataFrame(trainer.state.log_history)[['epoch', 'loss', 'eval_loss', 'eval_f1']]
df = df.rename(columns={'eval_loss':'Validation_loss',
                      'loss':'Training Loss', 'epoch':'Epoch', 'eval_f1':'F1'})
df['Epoch'] = df['Epoch'].apply(lambda x:round(x))
df['Training Loss'] = df['Training Loss'].ffill()
df[['Validation_loss','F1']] = df[['Validation_loss','F1']].bfill().ffill()
df.drop_duplicates()

In [ ]:
text_de = "Jack Dean ist ein Informatiker bei Google in Kallifornien"
tag_text(text_de, tags, trainer.model, xlmr_tokenizer)

# Error Analysis

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    batch = data_collator(features)
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    with torch.no_grad():
        output = trainer.model(input_ids, attention_mask)
        predicted_label = torch.argmax(output.logits, axis = -1).cpu().numpy()

        loss = cross_entropy(output.logits.view(-1, 7),
                             labels.view(-1), reduction = 'none')
        loss = loss.view(len(input_ids), -1).cpu().numpy()

        return {'loss': loss, 'predicted_label': predicted_label}

In [ ]:
valid_set = panx_de_encoded['validation']
valid_set = valid_set.map(forward_pass_with_label, batched = True, batch_size = 32)
df = valid_set.to_pandas()
df

In [ ]:
index2tag[-100] = 'IGN'

df['input_tokens'] = df['input_ids'].apply(lambda x: xlmr_tokenizer.convert_ids_to_tokens(x))
df['predicted_label'] = df['predicted_label'].apply(lambda x: [index2tag[i] for i in x])
df['labels'] = df['labels'].apply(lambda x: [index2tag[i] for i in x])
df['loss'] = df.apply(lambda x: x['loss'][:len(x['input_ids'])], axis = 1)
df['predicted_label'] = df.apply(lambda x: x['predicted_label'][:len(x['input_ids'])], axis = 1)
df.head(1)

In [ ]:
df_tokens = df.apply(pd.Series.explode)
df_tokens = df_tokens.query("labels != 'IGN'")
df_tokens['loss'] = df_tokens['loss'].astype(float).round(2)
df_tokens.head(7)

In [ ]:
(
    df_tokens.groupby('input_tokens')[['loss']]
    .agg(['count', 'mean', 'sum'])
    .droplevel(level = 0, axis = 1)
    .sort_values(by = 'sum', ascending = False)
    .reset_index()
    .head(10)
    .T
)

In [ ]:
(
    df_tokens.groupby('labels')[['loss']]
    .agg(['count', 'mean', 'sum'])
    .droplevel(level = 0, axis = 1)
    .sort_values(by = 'sum', ascending = False)
    .reset_index()
    .round(2)
    .T
)

In [ ]:
1-np.log(1/7) #bu qeder sehv ederdi eger hecne oyrenmeseydi

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize = 'true')
    fig, ax = plt.subplots(figsize = (6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = labels)
    disp.plot(cmap = plt.cm.Purples, values_format = '.2f', ax = ax, colorbar = False)
    plt.title("Confusion matrix")
    plt.show()

In [ ]:
plot_confusion_matrix(df_tokens['predicted_label'],
                      df_tokens['labels'], tags.names)

In [ ]:
def get_samples(df):
    for _, row in df.iterrows():
        labels, preds, tokens, losses = [], [], [], []
        for i, mask in enumerate(row['attention_mask']):
            if i not in {0, len(row['attention_mask']) - 1}:
                labels.append(row['labels'][i])
                preds.append(row['predicted_label'][i])
                tokens.append(row['input_tokens'][i])
                losses.append(f"{row['loss'][i]:.2f}")

        df_tmp = pd.DataFrame({'tokens': tokens, 'labels': labels,
                               'preds' : preds, 'losses': losses}).T
        yield df_tmp

df['total_loss'] = df['loss'].apply(sum)
df_tmp = df.sort_values(by = 'total_loss', ascending = False).head(3)

for sample in get_samples(df_tmp):
    display(sample)
#silver label - basga model terefinden labellanib
#golden label - insan terefinden labellanib

#Cross-Lingual Transfer

In [ ]:
def get_f1_score(trainer, dataset):
    return trainer.predict(dataset).metrics['test_f1']

In [ ]:
f1_scores = defaultdict(dict) #adi dict kimi, yoxdusa error qaytarmir
f1_scores['de']['de'] = get_f1_score(trainer, panx_de_encoded['test'])
print(f"F1-score of [de] model on [de] dataset: {f1_scores['de']['de']:.3f}")

In [ ]:
text_fr = "Jeff Dean est informaticien ches Google en Californie"
tag_text(text_fr, tags, trainer.model, xlmr_tokenizer)

In [ ]:
def evaluate_lang_performance(lang, trainer):
    panx_ds = encode_panx_dataset(panx_ch[lang])
    return get_f1_score(trainer, panx_ds['test'])

In [ ]:
f1_scores['de']['fr'] = evaluate_lang_performance('fr', trainer)
print(f"F1-score pf [de] model on [fr] dataset: {f1_scores['de']['fr']:.3f}")

In [ ]:
f1_scores['de']['it'] = evaluate_lang_performance('it', trainer)
print(f"F1-score pf [de] model on [it] dataset: {f1_scores['de']['it']:.3f}")

In [ ]:
f1_scores['de']['en'] = evaluate_lang_performance('en', trainer)
print(f"F1-score pf [de] model on [en] dataset: {f1_scores['de']['en']:.3f}")

In [ ]:
panx_fr_encoded = encode_panx_dataset(panx_ch['fr'])

#when does zero-shot transfer make sense?

In [ ]:
def train_on_subset(dataset, num_samples):
    train_ds=dataset['train'].shuffle(seed=42).select(range(num_samples))
    valid_ds=dataset['validation']
    test_ds=dataset['test']
    training_args.logging_steps=len(train_ds)//batch_size

    trainer=Trainer(model_init=model_init, args=training_args,
                    data_collator=data_collator, compute_metrics=compute_metrics,
                    train_dataset=train_ds, eval_dataset=valid_ds,
                    processing_class=xlmr_tokenizer)

    trainer.train()
    if training_args.push_to_hub:
        trainer.push_to_hub(commit_message="Training completed!")

    f1_score=get_f1_score(trainer, test_ds)
    return pd.DataFrame.from_dict({"num_samples": [len(train_ds)], "f1_score": [f1_score]})

In [ ]:
training_args.push_to_hub = False
metrics_df = train_on_subset(panx_fr_encoded, 250)
metrics_df

In [ ]:
for num_samples in [500, 1000, 2000, 4000]:
    metrics_df = pd.concat([metrics_df, train_on_subset(panx_fr_encoded, num_samples)],
                           ignore_index = True)

Epoch,Training Loss,Validation Loss,F1
1,1.426300,0.970359,0.317760
2,0.826400,0.656759,0.603703


Epoch,Training Loss,Validation Loss,F1
1,1.426300,0.970359,0.317760
2,0.826400,0.656759,0.603703
3,0.568100,0.556923,0.664750


Epoch,Training Loss,Validation Loss,F1
1,1.102700,0.530733,0.684364
2,0.473300,0.404460,0.734747


In [ ]:
fig, ax = plt.subplots()
ax.axhline(f1_score['de']['fr']ls = "--", color = "r")
metrics_df.set_index("num_samples").plot(ax = ax)
plt.legend(["Zero-shot from de", "Fine-tuned on fr"], loc = "lower right")
plt.ylim((0, 1))
plt.xlabel("Number of Training Samples")
plt.ylabel("F1 Score")
plt.show()

# Fine-tuning on Multiple Languages at once

In [ ]:
from datasets import concatenate_datasets

def concatenate_splits(corpora):
    multi_corpus = DatasetDict()
    for split in corpora[0].keys():
        multi_corpus[split] = concatenate_datasets(
            [corpus[split] for corpus in corpora]).shuffle(seed = 42)
    return multi_corpus

In [ ]:
panx_de_fr_encoded = concatenate_splits([panx_de_encoded, panx_fr_encoded])

In [ ]:
training_args.logging_steps = len(panx_de_fr_encoded['train']) // batch_size
training_args.push_to_hub = False
training_args.output_dir = "xlm-roberta-base-finetuned-panx-de-fr"

trainer = Trainer(model_init = model_init, args = training_args,
                  data_collator = data_collator, compute_metrics = compute_metrics,
                  preprocessing_class = xlmr_tokenizer, train_dataset = panx_de_fr_encoded['train'],
                  eval_dataset = panx_de_fr_encoded['validation'])
trainer.train()

In [ ]:
for lang in langs:
    f1 = evaluate_lang_performance(lang, trainer)
    print(f"F1-score of [de-fr] model on [{lang}] dataset: {f1:.3f}")